In [2]:
from src.data.load_data import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters

In [ ]:
ach_at_02 = load_MEA_data(folder = "data/raw/Ach-AT-subject-02")
# take middle horizontal strip of electrodes
label_MEA_data(ach_at_02, output='ach_at_subject_02')

In [3]:
df = pd.read_hdf("data/processed/ach_at_subject_02.h5")

In [5]:
y = (df[['window_id','y']]
     .drop_duplicates()
     .set_index('window_id')
     .T
     .squeeze()
     .sort_index(0))

cols = ['t', 'window_id', 15, 14, 13,  4, 57, 48, 47, 46]
df_middle = df[cols]
df = df.drop(columns = ['y'])

In [6]:
X = extract_features(df, column_id='window_id', column_sort='t', default_fc_parameters=MinimalFCParameters())
X.to_hdf('achat_02_min.h5', key = 'features', complevel = 9)

Feature Extraction: 100%|██████████| 10/10 [00:04<00:00,  2.01it/s]


In [7]:
X_middle = extract_features(df_middle, column_id='window_id', column_sort='t', default_fc_parameters=MinimalFCParameters())
X_middle.to_hdf('achat_02_min_middle.h5', key = 'features', complevel = 9)

Feature Extraction: 100%|██████████| 10/10 [00:00<00:00, 19.79it/s]


In [8]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

In [24]:
# first try with just the middle strip
impute(X_middle)
X_middle_filtered = select_features(X_middle, y)
X_middle_train, X_middle_test, y_train, y_test = train_test_split(X_middle_filtered, y, test_size=.4)
tree_middle = tree.DecisionTreeClassifier()
tree_middle.fit(X_middle_train, y_train)
print(classification_report(y_test, tree_middle.predict(X_middle_test)))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        13
           1       0.77      0.91      0.83        11
           2       0.85      0.79      0.81        14

    accuracy                           0.87        38
   macro avg       0.87      0.87      0.87        38
weighted avg       0.88      0.87      0.87        38



In [23]:
# next try with the entire electrode array
impute(X)
X_filtered = select_features(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)
tree_full = tree.DecisionTreeClassifier()
tree_full.fit(X_train, y_train)
print(classification_report(y_test, tree_full.predict(X_test)))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96        11
           1       1.00      0.62      0.77        16
           2       0.69      1.00      0.81        11

    accuracy                           0.84        38
   macro avg       0.87      0.88      0.85        38
weighted avg       0.89      0.84      0.84        38

